In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install -q -U "tensorflow-text==2.13.*"
!pip install "tf-models-official==2.13.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.7/479.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 21.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=51c6fb4e7e2a77284cf0a154c3a220c22e7353156f8d3de3e48c89b0125ace84
 

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
from official.nlp import optimization
import os
import pandas as pd

In [ ]:
dataset = pd.read_csv("/content/dataset_sn.csv")
dataset = dataset.drop('type', axis=1)

# Cetak dataset pelatihan
print("List Dataset:")
print(dataset)

List Dataset:
    label                                                 SN
0       S  Saya menyukai hal-hal yang konkret dan praktis...
1       S  Saya lebih memilih berfokus pada fakta dan log...
2       S  Saya menghargai kejujuran dan konsistensi, dan...
3       S  Saya lebih nyaman ketika memiliki jadwal yang ...
4       S  Saya suka mengamati hal-hal yang ada di sekita...
..    ...                                                ...
395     N  Saya lebih terbuka terhadap konsep abstrak, se...
396     N  Saya merasa terkoneksi dengan ide-ide yang bel...
397     N  Keputusan saya cenderung dipengaruhi oleh intu...
398     N  Tanpa perlu dipandu oleh petunjuk yang rinci, ...
399     N  Kesanggupannya untuk membaca ekspresi wajah da...

[400 rows x 2 columns]


In [ ]:
df = pd.DataFrame(dataset)
df['SN'] = df['SN'].str.strip()

# Membagi dataset berdasarkan kelompok
grouped = df.groupby('label')

# Inisialisasi DataFrame train dan valid
df_train = pd.DataFrame()
df_valid = pd.DataFrame()

# Memproses setiap kelompok
for name, group in grouped:
    # Menentukan jumlah data untuk validasi
    valid_size = int(len(group) * 0.2)

    # Memilih data untuk validasi
    valid_data = group.sample(n=valid_size, random_state=42)

    # Memilih data untuk train
    train_data = group.drop(valid_data.index)

    # Menggabungkan hasil pemisahan ke dalam DataFrame train dan valid
    df_train = pd.concat([df_train, train_data])
    df_valid = pd.concat([df_valid, valid_data])

#acak
df_train = df_train.sample(frac=1, random_state=42)
df_valid= df_valid.sample(frac=1, random_state=42)

# Cetak hasil
# print("Data Train:")
# print(df_train)
# print("\nData Valid:")
# print(df_valid)

In [ ]:
train_data = df_train
valid_data = df_valid
target_label = 'S' #label 1 Ekstrovert #label 1 Sensing #label 1 feeling #label 1 perceiving
label_train = (train_data['label'].str[0] == target_label).astype(int).to_numpy()
label_valid = (valid_data['label'].str[0] == target_label).astype(int).to_numpy()

In [ ]:
x_train = train_data['SN'].to_numpy() #isi sesuai kolom dataset
x_valid = valid_data['SN'].to_numpy()
# print(x_train.shape,x_valid.shape)
# # print(label_train)
# # print(x_train)

In [ ]:
encoder_url = 'https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/multi-cased-l-12-h-768-a-12/versions/4'
preprocess_url = "https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/multi-cased-preprocess/versions/3"
embedding_model1 = hub.KerasLayer(encoder_url, trainable=True)
preprocess_model1 = hub.KerasLayer(preprocess_url)

In [ ]:
@tf.keras.utils.register_keras_serializable(package='Custom', name='KerasLayer')
class KerasLayerWrapper(hub.KerasLayer):
    pass

def build_classifier_model():
    text_input1           = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Input_Sensing-Intuitive')
    preprocessing_layer1  = KerasLayerWrapper(preprocess_model1, name='Bert_Preprocessing-Sensing_Intuitive')
    encoder_inputs1       = preprocessing_layer1(text_input1)
    encoder1              = KerasLayerWrapper(embedding_model1, name='Bert-Encoder_Sensing-Intuitive',trainable=True)
    outputs1              = encoder1(encoder_inputs1)['pooled_output']
    outputs1              = tf.keras.layers.Dropout(0.5 , name="Dropout_Sensing-Intuitive")(outputs1)
    outputs1              = tf.keras.layers.Flatten(name = "Flatten-Sensing-Intuitive")(outputs1)
    outputs1              = tf.keras.layers.Reshape((1, -1), name = "Reshape-Sensing_Intuitive")(outputs1)
    outputs1              = tf.keras.layers.LSTM(64, return_sequences=True, name = "LSTM-1_Sensing-Intuitive")(outputs1)
    outputs1              = tf.keras.layers.LSTM(64, name = "LSTM-2_Sensing_Intuitive")(outputs1)
    outputs1              = tf.keras.layers.Dense(512, activation='relu', name = "Dense_1-Sensing_Intuitive")(outputs1)
    label1                = tf.keras.layers.Dense(1, activation='sigmoid', name='Classifier_Sensing-Intuitive')(outputs1)

    return tf.keras.Model(inputs=[text_input1], outputs=[label1])

In [ ]:
# def build_classifier_model():
#     text_input1 = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text1')

#     # Gunakan lapisan pre-processing terpisah untuk setiap input
#     preprocessing_layer1 = hub.KerasLayer(preprocess_model1, name='preprocessing1')
#     encoder_inputs1 = preprocessing_layer1(text_input1)
#     encoder1 = hub.KerasLayer(embedding_model1, name='BERT_encoder_1', trainable=True)
#     outputs1 = encoder1(encoder_inputs1)['pooled_output']
#     outputs1 = tf.keras.layers.Dropout(0.5 , name="dropout_1")(outputs1)
#     outputs1 = tf.keras.layers.Flatten()(outputs1)
#     outputs1 = tf.keras.layers.Reshape((1, -1))(outputs1)
#     outputs1 = tf.keras.layers.LSTM(64, return_sequences=True)(outputs1)
#     outputs1 = tf.keras.layers.LSTM(64)(outputs1)
#     outputs1 = tf.keras.layers.Dense(512, activation='relu')(outputs1)
#     label1 = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier1')(outputs1)

#     return tf.keras.Model(inputs=[text_input1], outputs=[label1])


In [ ]:
model = build_classifier_model()
# model.summary()

In [ ]:
model.load_weights("/content/drive/MyDrive/Colab Notebooks/sensing_intuitive_weights.h5")

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/model_sensing_intuitive")

In [ ]:
epochs = 100
steps_per_epoch = 16
batch_size = 2
optimizer = tf.keras.optimizers.Adam(learning_rate=0.000001)

In [ ]:
# !pip install -q tf-models-official

In [ ]:
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),  # Fungsi kerugian untuk klasifikasi biner
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['loss'] <= 0.1 and logs['val_loss'] <= 0.1:
            self.model.stop_training = True

# Path untuk menyimpan checkpoint pada setiap epoch
checkpoint_path_epoch = "/content/model/model_checkpoint_epoch.h5"

# Path untuk menyimpan checkpoint model terbaik
checkpoint_path_best = "/content/model/model_checkpoint_best.h5"

# Callback untuk menyimpan checkpoint pada setiap epoch
checkpoint_callback_epoch = ModelCheckpoint(
    filepath=checkpoint_path_epoch,
    save_weights_only=True,
    save_best_only=False,  # Set ke False agar menyimpan pada setiap epoch
    verbose=1
)

# Callback untuk menyimpan checkpoint model terbaik
checkpoint_callback_best = ModelCheckpoint(
    filepath=checkpoint_path_best,
    save_weights_only=True,
    save_best_only=True,  # Set ke True agar hanya menyimpan model terbaik
    monitor='val_loss',
    mode='min',
    verbose=1
)

In [ ]:
# untuk lanjut
model.fit(
    x=x_train, y=label_train,
    epochs=epochs,
    validation_data=(x_valid, label_valid),batch_size=batch_size,
    callbacks=[checkpoint_callback_epoch,checkpoint_callback_best, CustomCallback()]
)

Epoch 1/100
160/160 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.5375
Epoch 1: saving model to /content/model/model_checkpoint_epoch.h5

Epoch 1: val_loss improved from inf to 0.69285, saving model to /content/model/model_checkpoint_best.h5
160/160 [==============================] - 27s 167ms/step - loss: 0.6928 - accuracy: 0.5375 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 2/100
160/160 [==============================] - ETA: 0s - loss: 0.6927 - accuracy: 0.5469
Epoch 2: saving model to /content/model/model_checkpoint_epoch.h5

Epoch 2: val_loss improved from 0.69285 to 0.69252, saving model to /content/model/model_checkpoint_best.h5
160/160 [==============================] - 29s 183ms/step - loss: 0.6927 - accuracy: 0.5469 - val_loss: 0.6925 - val_accuracy: 0.5125
Epoch 3/100
160/160 [==============================] - ETA: 0s - loss: 0.6925 - accuracy: 0.5437
Epoch 3: saving model to /content/model/model_checkpoint_epoch.h5

Epoch 3: val_loss improved f

In [ ]:
model.save_weights("/content/drive/MyDrive/Colab Notebooks/sensing_intuitive_weights.h5")

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/model_sn/")

In [ ]:
# sensing == 1
model.predict(["senang menyimak keadaan sekitarnya dengan seksama. Setiap hari, ia akan memperhatikan hal-hal kecil yang mungkin terlewatkan oleh orang lain. Misalnya, ia selalu tahu kapan tetangganya mengganti bunga di halaman depan rumahnya atau ketika ada perubahan kecil dalam rutinitas sehari-hari di kafe yang sering ia kunjungi. Rudi juga cenderung menanggapi kebutuhan orang lain tanpa diminta, seperti membantu tetangga membawa belanjaan atau memberi tempat duduk kepada seorang ibu hamil di bus. Dengan kepribadiannya yang penuh perhatian terhadap sekitarnya, Rudi menjalin hubungan yang kuat dengan komunitasnya dan selalu memberikan sentuhan manusiawi pada setiap momen sehari-hari."])

1/1 [==============================] - 2s 2s/step


array([[0.9440896]], dtype=float32)

In [ ]:
# intuitive == 0
model.predict(["memiliki kecenderungan untuk fokus pada ide-ide besar, potensi, dan konsep-konsep abstrak. Mereka melihat gambaran keseluruhan dan mencari makna di balik informasi yang mereka terima. Orang dengan preferensi intuisi dalam tipe kepribadian Myers-Briggs mungkin cenderung lebih suka berpikir tentang masa depan dan memiliki kepekaan terhadap peluang dan kemungkinan. Mereka sering menunjukkan kreativitas, daya khayal yang tinggi, dan ketertarikan pada ide-ide inovatif"])

1/1 [==============================] - 0s 68ms/step


array([[0.04995943]], dtype=float32)

In [ ]:
# Contoh: Memuat model SavedModel
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/model_sn")

# Sekarang, Anda dapat menggunakan loaded_model seperti model lainnya
# Misalnya: melakukan prediksi

predictions = loaded_model.predict(["senang menyimak keadaan sekitarnya dengan seksama. Setiap hari, ia akan memperhatikan hal-hal kecil yang mungkin terlewatkan oleh orang lain. Misalnya, ia selalu tahu kapan tetangganya mengganti bunga di halaman depan rumahnya atau ketika ada perubahan kecil dalam rutinitas sehari-hari di kafe yang sering ia kunjungi. Rudi juga cenderung menanggapi kebutuhan orang lain tanpa diminta, seperti membantu tetangga membawa belanjaan atau memberi tempat duduk kepada seorang ibu hamil di bus. Dengan kepribadiannya yang penuh perhatian terhadap sekitarnya, Rudi menjalin hubungan yang kuat dengan komunitasnya dan selalu memberikan sentuhan manusiawi pada setiap momen sehari-hari."])

print(predictions)

1/1 [==============================] - 2s 2s/step
[[0.9440896]]


In [ ]:
# Mendapatkan daftar perangkat yang tersedia (CPU dan GPU)
devices = tf.config.list_physical_devices()

if len(devices) == 0:
    print("Tidak ada perangkat fisik yang ditemukan.")
else:
    print("Perangkat yang tersedia:")
    for device in devices:
        print(f"- {device.name} ({device.device_type})")

In [ ]:
print(model.predict(["sayap pergi sangat mudah temukan berhubungan setiap enneagram infj langka mustahil tidak mungkin semuanya mungkin tergantung naikkan cuure dll menang temukan banyak enneagram individualis di korea utara contoh mungkin kamp konsentrasi mh tebak akan membantu akan menemukan dosa mematikan lihat masalah lol iri sangat jelas tidak puas salah satu favorit katakan agak suka menyalin mekanisme seperti tidak ada hal semuanya akan baik-baik saja mungkin berpengaruh dengan baik semoga memperbaiki kerakusan di bawah kendali kecuali menggoda rumah coklat makan semuanya suatu hari gulma merokok tidak ada yang meninggalkan orang yang terlalu marah berarti kencing lelucon inti mempengaruhi banyak prolly w bermain dengan baik seseorang sesuatu yang terbelakang cara perlahan-lahan membuat gila seperti tahu seperti hei mengemudi kayu tengah tidak ada tempat duduk biarkan mengambil roda mobil duduk menang dapat mengemudi lagi hei paling tidak bisa duduk hahahahah ieriozuepoanhceiphfiozejfijrioe berhenti contoh buruk cukup mudah-mudahan memperbaiki fokus pikiran yang sadar akan sangat membantu dengan baik mungkin membutuhkan waktu pada akhirnya menjadi jelas kirim sesuatu mungkin perlu perubahan tahu partikal diskrit berpikir sebagian besar berpikir cara terhitung banyaknya bayangkan paling tidak mendekati sesuatu yang diskrit bahkan seperti fungsi tidak pernah cukup konvergen masalah fungsi kontinu topps mengurutkan basis deteksi emulasi menggunakan perangkat lunak tertentu menggunakan tujuan perangkat lunak bahkan meniru akan memicu perma larangan membeli paket skema baru-baru ini berharap satu opsi izinkan giliran animasi ingin mengenai paket kartu tambahkan stok perlu melihat paket terbuka pasti ingin melihat sumber daya jelek hit ringan terjadi kartu pertama oleh karena itu tutup waktu paket berhenti buka paket paling tidak akan cepat secara teoritis topps dapat memperkenalkan opsi pembelian mui akan sangat buruk aplikasi setidaknya tombol pembelian paket manual lambat pembeli besar pembeli kecil dapatkan paket populer jika tidak, paus dapat sepenuhnya membeli kartu satu pembelian tahu benar berani orang berpikir percaya hal mengatakan dunia pukulan tongkat mengapung selamanya ruang membekukan bola mati lemas mampu membuat kesepakatan kesengsaraan dua secara profesional mengakhiri kebencian cocok stereotip buruk keras suka memerintah kaku tak kenal ampun menurunkan semangat menjengkelkan satu mitra bisnis bertarung dengan keras secara agresif menunggu satu sama lain departemen pekerjaan yang berbeda mempengaruhi pekerjaan orang menjengkelkan yang menyedihkan membuat semua orang di sekitar menderita harapan bertemu dengan contoh yang baik sosiopat masa depan imo mengatakan dengan tepat satu akan mengatakan edit oh terima kasih Tuhan semua orang mendapatkan keramahan yang rendah jujur ?de?????de????menang mudah intj entj fungsi urutan yang berbeda intjs seperti banyak introvert sering memimpin fungsi tambahan situasi sosial perbedaan besar entjs kebutuhan tinggi menyelesaikan hal intjs kebutuhan tinggi secara akurat memahami intjs datang menggigit lalai ni biarkan kepala hidup gunakan te secara proaktif menjaga lebih memilih dukungan lingkungan ni mampu menjaga tatanan lingkungan ideal dengan benar mengarahkan energi ke dalam entj gunakan ni melayani te gunakan refleksi prediksi memesan tujuan satu cara mungkin membedakan reaksi gangguan lingkungan intj menjengkelkan seperti perbaikan hebat mendapatkan kembali hal yang sangat peduli entj oportunistik berpikir peluang besar mengatur ulang cara pasti enfps cinta istjs baik gender teman enfps kreativitas semangat hidup menemukan sangat menarik sensitivitas bebas semangat keseimbangan dingin kekakuan tambahan fi tersier te temukan di antara orang-orang yang sangat memahami nilai pandangan dunia bahkan setuju meskipun istjs berbagi fungsi sering menemukan tangkapan membatasi si konotasi teman monster monster berpikir pengungkapan penuh entj bisnis pesanan biasa yaitu menginjak-injak menggambarkan monster seperti romantisisasi yang tidak perlu sama sekali sarung pedang dll melayani gangguan mungkin mengambil fakta berhenti memberi tahu op monster"]))

In [ ]:
from google.colab import drive
import zipfile
import os

# Path folder yang ingin di-zip
folder_path = '/content/drive/MyDrive/Colab Notebooks/model_sensing_intuitive'

# Path hasil zip
output_zip_path = '/content/drive/MyDrive/Colab Notebooks/model_sn.zip'

# Fungsi untuk zip folder
def zip_folder(folder_path, output_zip_path):
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

# Memanggil fungsi untuk zip folder
zip_folder(folder_path, output_zip_path)